In [245]:
%load_ext autoreload
%autoreload 2
import numpy as np
import qutip
import theano
import theano.tensor as T
import QubitNetwork

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [224]:
qutip.tensor([qutip.basis(2, 0) for _ in range(4)])
foo = qutip.rand_ket(N=16)
foo.dims = [[2, 2, 2, 2], [1, 1, 1, 1]]
np.random.random()

Quantum object: dims = [[2, 2, 2, 2], [1, 1, 1, 1]], shape = (16, 1), type = ket
Qobj data =
[[-0.27955145-0.25388861j]
 [ 0.11561937-0.07440324j]
 [ 0.30239505-0.15887813j]
 [-0.30372354-0.09205277j]
 [-0.28287221+0.04932048j]
 [ 0.02521226-0.06136463j]
 [ 0.17219526-0.02771928j]
 [ 0.23077973+0.1066017j ]
 [ 0.08238318+0.02112077j]
 [-0.09055917-0.00271109j]
 [-0.05852606-0.05723796j]
 [ 0.30835457-0.14476351j]
 [ 0.19813409-0.21939289j]
 [-0.26818802-0.09973814j]
 [ 0.27762714-0.20017743j]
 [ 0.02145614-0.1188905j ]]

In [261]:
eta = 0.8182
xi = 0.0587
initial_ancilla = qutip.Qobj([[np.cos(eta)], [np.sin(eta) * np.exp(1j * xi)]])

net = QubitNetwork.QubitNetwork(
    num_qubits=4,
    interactions=('all', ['xx', 'zz']),
    self_interactions=('all', ['x', 'z']),
    ancillas_state=initial_ancilla,
    system_qubits=[0, 1, 2]
)

factor  = -8.940 / 4 * net.tuple_to_xs_factor(((0, 1), 'zz'))
factor += -4.957 / 4 * net.tuple_to_xs_factor(((0, 2), 'zz'))
factor += -4.957 / 4 * net.tuple_to_xs_factor(((1, 2), 'zz'))
factor += -5.657 / 4 * net.tuple_to_xs_factor(((0, 3), 'zz'))
factor += -5.657 / 4 * net.tuple_to_xs_factor(((1, 3), 'zz'))

factor += 15.060 / 4 * net.tuple_to_xs_factor(((2, 3), 'xx'))

factor += -2.428 / 2 * net.tuple_to_xs_factor((0, 'z'))
factor += -2.428 / 2 * net.tuple_to_xs_factor((1, 'z'))
factor += -4.957 / 2 * net.tuple_to_xs_factor((2, 'z'))
factor += -0.165 / 2 * net.tuple_to_xs_factor((3, 'z'))

factor += -19.08 / 2 * net.tuple_to_xs_factor((2, 'x'))
factor += -4.297 / 2 * net.tuple_to_xs_factor((3, 'x'))

foo = utils.bigreal2complex(factor)
foo = qutip.Qobj(foo)
foo.expm()

Quantum object: dims = [[16], [16]], shape = (16, 16), type = oper, isherm = False
Qobj data =
[[  9.85736656e-01-0.1335818j    2.99768276e-03-0.023827j
   -3.78613376e-03-0.06712283j   9.49133837e-05+0.07336633j
    0.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j
    0.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j
    0.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j
    0.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j        ]
 [  2.99768276e-03-0.023827j     9.94792434e-01-0.01985275j
   -7.97248347e-04+0.05816631j   3.75639629e-03-0.07758288j
    0.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j
    0.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j
    0.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j
    0.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j        ]
 

In [198]:
net = QubitNetwork.QubitNetwork(num_qubits=4,
                           interactions=('all', ['yy', 'zz']),
                           self_interactions=('all', ['x']),
                           system_qubits=[0, 1, 2])
random_Js = np.random.randn(net.num_interactions + net.num_self_interactions)
nice_hs = np.asarray([1j * utils.bigreal2complex(big_h) for big_h in net.hs_factors])
nice_Js = np.asarray([1j * utils.bigreal2complex(big_J) for big_J in net.Js_factors])
nice_factors = np.concatenate((nice_hs, nice_Js), axis=0)
H = qutip.Qobj(np.tensordot(random_Js, nice_factors, axes=1))
expH = (-1j * H).expm()
expH

Quantum object: dims = [[16], [16]], shape = (16, 16), type = oper, isherm = False
Qobj data =
[[ 0.15262938+0.0096427j   0.11689735-0.10811018j -0.15161546+0.2194946j
   0.10106818+0.03663583j -0.27380367+0.14628458j  0.01580962+0.01612412j
   0.07257164+0.14756418j  0.05573154+0.03332085j  0.25616656+0.00453465j
  -0.17703720+0.02344678j -0.21084282+0.18366693j -0.14921499-0.35904427j
  -0.44599860+0.08656022j  0.10051142+0.10537405j -0.11470614+0.07247606j
   0.40892314-0.01608505j]
 [ 0.11689735-0.10811018j  0.11985232+0.41732035j  0.09037383-0.17923525j
   0.10878362+0.04996035j -0.16768269+0.02969284j -0.09233240-0.13948621j
   0.09027606+0.01655867j -0.15279176+0.10715808j  0.02317725-0.14170529j
   0.22900764+0.24594528j -0.31647426+0.23005299j  0.16635366-0.12315148j
  -0.03986231+0.20901533j -0.30402343-0.11053746j  0.36488864+0.01789606j
  -0.11470614+0.07247606j]
 [-0.15161546+0.2194946j   0.09037383-0.17923525j  0.20085609-0.1874757j
  -0.17604253-0.19734373j -0.21317330-0

In [156]:
# theano.config.optimizer = 'None'
net = QubitNetwork.QubitNetwork(num_qubits=4,
                           interactions=('all', ['yy', 'zz']),
                           self_interactions=('all', ['x']),
                           system_qubits=[0, 1, 2])
QubitNetwork.sgd_optimization(
#     net='nets/xyz_net.pickle',
    net=net,
    learning_rate=1,
    n_epochs=100,
    batch_size=10
#     backup_file='nets/main_yz.pickle'
)

Building the model...
Let's roll!
Epoch 0, 0.10320570431884973
Epoch 1, 0.12357493369444314
Epoch 2, 0.12786824291535034
Epoch 3, 0.13165989246433848
Epoch 4, 0.143210899179843
Epoch 5, 0.15214264944393427
Epoch 6, 0.1523200801260625
Epoch 7, 0.15234078914031648
Epoch 8, 0.15234351862407527
Epoch 9, 0.15234403087814785
Epoch 10, 0.15234414432883717
Epoch 11, 0.15234417069770395
Epoch 12, 0.15234417690066357
Epoch 13, 0.15234417836403713
Epoch 14, 

KeyboardInterrupt: 

In [143]:
net = QubitNetwork.QubitNetwork(num_qubits=3,
                           interactions=('all', ['yy', 'zz']),
                           self_interactions=('all', ['x', 'z']),
                           system_qubits=[0, 1])
net_topology = {
    ((0, 1), 'yy'): 'a',
    ((0, 1), 'zz'): 'a',
    (0, 'z'): 'b'
}
# net.tuple_to_xs_factor(((1, 2), 'zz'))
foo = net.build_custom_H_factors(net_topology)
qutip.Qobj(-1j * utils.bigreal2complex(foo[0]))
qutip.Qobj(-1j * utils.bigreal2complex(foo[1]))
qutip.Qobj(-1j * utils.bigreal2complex(net.hs_factors[0]))

symbols: ['a', 'b']


Quantum object: dims = [[8], [8]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]]

In [15]:
QubitNetwork.sgd_optimization(
    net='net_on_training.pickle',
    learning_rate=0.02,
    n_epochs=100,
    batch_size=10,
    backup_file='nets/main.pickle'
)

Network backup saved in nets/main.pickle
Building the model...
Let's roll!
Epoch 0, 0.10090931232365992
Epoch 1, 0.1009372679211838
Epoch 2, 0.1009546087246383
Epoch 3, 0.10096514284554362
Epoch 4, 0.10097118764200791
Epoch 5, 0.10097427746634965
Epoch 6, 0.10097548292369664
Epoch 7, 0.10097557392685576
Epoch 8, 0.10097511230671824
Epoch 9, 0.10097450947887666
Epoch 10, 0.10097406514681927
Epoch 11, 0.10097399476332956
Epoch 12, 0.10097444976317041
Epoch 13, 0.10097553282674948
Epoch 14, 0.10097730955681081
Epoch 15, 0.1009798174795236
Epoch 16, 0.10098307300822419
Epoch 17, 0.10098707683643891
Epoch 18, 0.10099181811117516
Epoch 19, 0.10099727765521979
Epoch 20, 0.10100343044648125
Epoch 21, 0.1010102475164909
Epoch 22, 0.10101769739489623
Epoch 23, 0.10102574719942538
Epoch 24, 0.10103436344946162
Epoch 25, 0.10104351266464962
Epoch 26, 0.10105316179683128
Epoch 27, 0.10106327853328508
Epoch 28, 0.10107383150110606
Epoch 29, 0.10108479039615667
Epoch 30, 0.10109612605495669
Epoch 31,